In [2]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#save excel file, open in notepad and save with utf-8 encoding. later, open txt file and save as .csv with utf-8 encoding
df = pd.read_csv("law2.csv")

In [4]:
df.head()

,article_id,article_text,source
0,1,Apple Inc supplier TSMC has counter-sued small...,https://www.reuters.com/article/db-tsmc-global...
1,2,"After weeks of scandal and speculation, the be...",https://www.theguardian.com/books/2019/apr/25/...
2,3,Adidas has been unsuccessful in an attempt to ...,https://www.theguardian.com/law/2019/jun/19/ad...
3,4,"Meghan, Duchess of Sussex has taken the unusua...",https://www.theguardian.com/uk-news/2019/oct/0...


In [5]:
df['article_text'][0]

'Apple Inc supplier TSMC has counter-sued smaller contract chipmaking rival GlobalFoundries Inc in the United States, Germany and Singapore, saying the U.S. firm has infringed 25 of its patents. Taiwan Semiconductor Manufacturing Co Ltd (TSMC) said that in lawsuits filed on Monday it has asked for injunctions to stop GlobalFoundries’ manufacture and sale of chips that infringe the patents. TSMC, the world’s largest contract chipmaker, said in a statement on Tuesday it was seeking “substantial monetary damages from GlobalFoundries” but did not specify an amount. The lawsuits come just over a month after GlobalFoundries sued TSMC for allegedly infringing 16 patents, seeking to stop imports into the United States and Germany of products made with technologies using those patents. “TSMC has long used its dominant market position to exert pressure on its smaller competitors, and the retaliatory litigation filed today is in keeping with that history,” Santa Clara, California-based GlobalFoun

In [6]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [7]:
sentences[:5]

['Apple Inc supplier TSMC has counter-sued smaller contract chipmaking rival GlobalFoundries Inc in the United States, Germany and Singapore, saying the U.S. firm has infringed 25 of its patents.',
 'Taiwan Semiconductor Manufacturing Co Ltd (TSMC) said that in lawsuits filed on Monday it has asked for injunctions to stop GlobalFoundries’ manufacture and sale of chips that infringe the patents.',
 'TSMC, the world’s largest contract chipmaker, said in a statement on Tuesday it was seeking “substantial monetary damages from GlobalFoundries” but did not specify an amount.',
 'The lawsuits come just over a month after GlobalFoundries sued TSMC for allegedly infringing 16 patents, seeking to stop imports into the United States and Germany of products made with technologies using those patents.',
 '“TSMC has long used its dominant market position to exert pressure on its smaller competitors, and the retaliatory litigation filed today is in keeping with that history,” Santa Clara, California

In [8]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [9]:
len(word_embeddings)

400000

In [10]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [11]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [13]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [14]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [15]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [16]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [17]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [18]:
from sklearn.metrics.pairwise import cosine_similarity


In [19]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [20]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [21]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


In [22]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

The company did not “prove that that mark has acquired, throughout the territory of the EU, distinctive character following the use which had been made of it”, the general court of the EU said on Wednesday.
He added that the case highlighted the challenge for well-known brands trying to gain protection across the EU.
“This ruling is limited to this particular execution of the three-stripe mark and does not impact on the broad scope of protection that Adidas has on its well-known three-stripe mark in various forms in Europe,” the company said in a statement.
“TSMC has long used its dominant market position to exert pressure on its smaller competitors, and the retaliatory litigation filed today is in keeping with that history,” Santa Clara, California-based GlobalFoundries said in a statement.
Mark Caddle, a trademark lawyer at the European intellectual property firm Withers & Rogers, said Adidas had “failed to provide sufficient evidence to show that when seeing three stripes on clothin